In [1]:
pip install timm

     |████████████████████████████████| 431 kB 4.2 MB/s 


In [2]:
import os
import numpy as np
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets
import matplotlib.pyplot as plt
from skimage import io, transform
from torchvision import transforms, utils
import timm
import torch.nn as nn
from sklearn.metrics import accuracy_score
import time

device = torch.device('cuda')
device

device(type='cuda')

In [3]:
class CustomImageDataset(Dataset):
  def __init__(self, annotations_file, img_dir, transform=None, target_transform=None, mode='train'):
    self.img_labels = pd.read_csv(annotations_file)
    self.img_dir = img_dir
    self.transform = transform
    self.target_transform = target_transform
    self.mode = mode

  def __len__(self):
    return len(self.img_labels)

  def __getitem__(self, idx):
    img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
    image = read_image(img_path)
    if self.mode == 'train':
      label = self.img_labels.iloc[idx, 1]
    else:  # test set has no label
      label = 'temp'
    if self.transform:
      image = self.transform(image)
    if self.target_transform:
      label = self.target_transform(label)
    return image, label

In [4]:
label_csv = '/content/drive/MyDrive/ZINDI-image/csv_files/Train.csv'
train_dir = '/content/drive/MyDrive/ZINDI-image/train'

train_ds = CustomImageDataset(annotations_file=label_csv,
                              img_dir=train_dir, 
                              transform=transforms.Compose([
                                                            transforms.ToPILImage(),
                                                            transforms.Resize((224,224)),
                                                            transforms.ToTensor(),
                                                            transforms.Normalize([0.44280738, 0.5332802, 0.33625707], 
                                                                                [0.17611444, 0.1734216, 0.17963623])
                                           ])
                              )
train_size = int(len(train_ds) * 0.8)
valid_size = len(train_ds) - train_size
train_ds, val_ds = random_split(train_ds,[train_size, valid_size])
train_dataloader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4)
val_dataloader = DataLoader(val_ds, batch_size=32, shuffle=True, num_workers=4)

In [5]:
class Network(nn.Module):
  def __init__(self):
    super(Network, self).__init__()
    self.model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=2)
        
  def forward(self, x):
    x = self.model(x)
    return x

model = Network().to(device)
# print(model)

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b0_ra-3dd342df.pth


In [6]:
def score_function(pred, real):
  score = accuracy_score(real, pred)
  return score

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()  # (pred, target)
scaler = torch.cuda.amp.GradScaler()

epochs = 30
min_val_loss = np.inf
early_trigger = 0  # for early stopping
patience = 5

for e in range(epochs):
  START=time.time()

  train_loss = 0.0
  train_pred=[]
  train_target=[]
  model.train()
  for data, target in train_dataloader:
    # Transfer Data to GPU if available
    if torch.cuda.is_available():
      data, target = data.cuda(), target.cuda()
    # Clear the gradients
    optimizer.zero_grad()
    # Forward Pass
    with torch.cuda.amp.autocast():
      pred = model(data)
      # Find the Loss
      loss = criterion(pred, target)
    # Calculate gradients
    scaler.scale(loss).backward()
    # Update Weights
    scaler.step(optimizer)
    scaler.update()
    # Calculate Loss
    train_loss += loss.item()/len(train_dataloader)
      
    train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
    train_target += target.detach().cpu().numpy().tolist()
        
  train_acc = score_function(train_pred, train_target)
    
  val_loss = 0.0
  val_pred=[]
  val_target=[]
  model.eval()
  with torch.no_grad():  # because we don't need grad for validation
    for data, target in val_dataloader:
      if torch.cuda.is_available():
        data, target = data.cuda(), target.cuda()
      # Forward Pass
      with torch.cuda.amp.autocast():
        pred = model(data)
        # Find the Loss
        loss = criterion(pred, target)
      # Calculate Loss
      val_loss += loss.item()/len(val_dataloader)

      val_pred += pred.argmax(1).detach().cpu().numpy().tolist()
      val_target += target.detach().cpu().numpy().tolist()

  val_acc = score_function(val_pred, val_target)

  TIME = time.time() - START
  print(f'Epoch {e+1}/{epochs} \t time : {TIME:.2f}s')
  print(f'Training Loss: {train_loss:.5f} \t\t Validation Loss: {val_loss:.5f}')
  print(f'Training accuracy: {train_acc:.5f} \t\t Validation accuracy: {val_acc:.5f}')
  if min_val_loss > val_loss:
    early_trigger = 0
    print(f'Validation Loss Decreased({min_val_loss:.5f}--->{val_loss:.5f}) \t Saving The Model')
    min_val_loss = val_loss
    # Saving State Dict
    torch.save(model.state_dict(), '/content/drive/MyDrive/ZINDI-image/notebooks/PT/saved/saved_model.pth')
  else:
    early_trigger += 1
  
  # early stopping
  if early_trigger >= patience:  
    break

Epoch 1/30 	 time : 310.28s
Training Loss: 1.05492 		 Validation Loss: 0.53374
Training accuracy: 0.86564 		 Validation accuracy: 0.97222
Validation Loss Decreased(inf--->0.53374) 	 Saving The Model
Epoch 2/30 	 time : 11.45s
Training Loss: 0.12723 		 Validation Loss: 0.14774
Training accuracy: 0.97452 		 Validation accuracy: 0.97531
Validation Loss Decreased(0.53374--->0.14774) 	 Saving The Model
Epoch 3/30 	 time : 11.60s
Training Loss: 0.04306 		 Validation Loss: 0.08892
Training accuracy: 0.98687 		 Validation accuracy: 0.97531
Validation Loss Decreased(0.14774--->0.08892) 	 Saving The Model
Epoch 4/30 	 time : 11.74s
Training Loss: 0.02841 		 Validation Loss: 0.03975
Training accuracy: 0.99151 		 Validation accuracy: 0.98457
Validation Loss Decreased(0.08892--->0.03975) 	 Saving The Model
Epoch 5/30 	 time : 11.61s
Training Loss: 0.01806 		 Validation Loss: 0.07618
Training accuracy: 0.99459 		 Validation accuracy: 0.97840
Epoch 6/30 	 time : 11.58s
Training Loss: 0.03208 		 Valid